In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import vectorbt as vbt

In [2]:
df = pd.read_csv('data/BTCUSDT_3h_2020-2025.csv')
dfs = {'BTCUSDT_2020-2025_trend_following': df}

In [3]:
# parameters_ranges = {
#     'regression_window': np.arange(10, 101, 10),
#     'trend_strength': np.linspace(1.0, 2.5, 6),
#     'vol_threshold': np.linspace(1, 2, 6)
# }
# parameters_ranges

In [5]:
from walk_forward_optimization import WalkForwardOptimizer


wfo = WalkForwardOptimizer(
    data=dfs, 
    param_ranges = {
        'regression_window': np.arange(6, 25, 2),
        'trend_strength': np.linspace(0.1, 3, 10),
        'vol_threshold': np.linspace(0.1, 3, 10)
    },
    n_windows=10,
    in_sample_percentage=0.7,
    allow_shorting=True,
    allow_leverage=False, # dataframe has to have df['leverage'] column to use leverage
    timeframe='3h',
    fees=0.0005,
    init_cash=100000,
    auto_select=True,
    trades_share_ratio_selection=True, # if true, not only sharpe is considered to pick best in-sample result, but also the number of trades.
    results_dir='twap_results'
)


In [6]:
wfo.optimize()


Processing pair: BTCUSDT_2020-2025_trend_following


2025-04-17 15:55:56,667 - Processing combination 10/1000
2025-04-17 15:55:56,669 - Processing combination 20/1000
2025-04-17 15:55:56,669 - Processing combination 30/1000
2025-04-17 15:55:57,475 - Processing combination 40/1000
2025-04-17 15:55:57,476 - Processing combination 50/1000
2025-04-17 15:55:57,477 - Processing combination 60/1000
2025-04-17 15:55:57,532 - Processing combination 70/1000
2025-04-17 15:55:57,534 - Processing combination 80/1000
2025-04-17 15:55:57,535 - Processing combination 90/1000
2025-04-17 15:55:59,082 - Processing combination 100/1000
2025-04-17 15:55:59,084 - Processing combination 110/1000
2025-04-17 15:55:59,084 - Processing combination 120/1000
2025-04-17 15:55:59,354 - Processing combination 130/1000
2025-04-17 15:55:59,356 - Processing combination 140/1000
2025-04-17 15:55:59,357 - Processing combination 150/1000
2025-04-17 15:55:59,453 - Processing combination 160/1000
2025-04-17 15:55:59,455 - Processing combination 170/1000
2025-04-17 15:55:59,455

{'BTCUSDT_2020-2025_trend_following': [{'window': 0,
   'in_sample_results': [{'params': {'regression_window': np.int64(8),
      'trend_strength': np.float64(0.1),
      'vol_threshold': np.float64(0.422)},
     'sharpe_ratio': 2.884455482448706,
     'sortino_ratio': 5.713156784508717,
     'calmar_ratio': 14.836736364291735,
     'total_return': 1.529387733652926,
     'max_drawdown': 0.10999789507593696,
     'total_trades': 192,
     'stats': Start                         1970-01-01 00:00:00.000000007
     End                           1970-01-01 00:00:00.000002806
     Period                                    350 days 00:00:00
     Start Value                                        100000.0
     End Value                                     252938.773365
     Total Return [%]                                 152.938773
     Benchmark Return [%]                             185.489927
     Max Gross Exposure [%]                                100.0
     Total Fees Paid             

In [ ]:
# import matplotlib.pyplot as plt

# # Set up the figure and axes
# plt.figure(figsize=(14, 8))

# # Plot the Close price
# plt.plot(df.index, df['Close'], label='Close Price', color='blue', alpha=0.6)

# # Highlight Trend-Following Regime
# plt.fill_between(
#     df.index,
#     df['Close'],
#     where=(df['Regime'] == 'Trend'),
#     color='red',
#     alpha=0.3,
#     label='Trend-Following Regime'
# )

# # Highlight Mean-Reverting Regime
# plt.fill_between(
#     df.index,
#     df['Close'],
#     where=(df['Regime'] == 'Mean-Revert'),
#     color='green',
#     alpha=0.3,
#     label='Mean-Reverting Regime'
# )

# # Add labels, legend, and title
# plt.title('Market Regimes Based on Volatility Filter', fontsize=16)
# plt.xlabel('Date/Time', fontsize=14)
# plt.ylabel('Close Price', fontsize=14)
# plt.legend(loc='upper left', fontsize=12)
# plt.grid(alpha=0.3)
# plt.tight_layout()

# # Show the plot
# plt.show()
